In [ ]:
import pandas as pd

# Load the dataset provided by the user
file_path = '/mnt/data/dataset'

# Read the dataset
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()


In [ ]:
# Rename the columns for better understanding
cleaned_data.columns = ["Date", "Adj Close", "Close", "High", "Low", "Open", "Volume"]

# Convert the "Date" column to datetime format
cleaned_data["Date"] = pd.to_datetime(cleaned_data["Date"])

# Set "Date" as the index for easier analysis
cleaned_data.set_index("Date", inplace=True)

# Display the cleaned dataset
cleaned_data.head()


In [ ]:
import ta  # Importing the technical analysis library for easier feature calculations

# Calculate 10-day and 50-day Simple Moving Averages
cleaned_data['ma_10'] = cleaned_data['Close'].rolling(window=10).mean()
cleaned_data['ma_50'] = cleaned_data['Close'].rolling(window=50).mean()

# Calculate Relative Strength Index (RSI) with a 14-day window
cleaned_data['rsi_14'] = ta.momentum.RSIIndicator(close=cleaned_data['Close'], window=14).rsi()

# Calculate Bollinger Bands (20-day MA ± 2σ)
bb_indicator = ta.volatility.BollingerBands(close=cleaned_data['Close'], window=20, window_dev=2)
cleaned_data['bb_upper'] = bb_indicator.bollinger_hband()
cleaned_data['bb_lower'] = bb_indicator.bollinger_lband()

# Calculate Moving Average Convergence Divergence (MACD)
macd_indicator = ta.trend.MACD(close=cleaned_data['Close'])
cleaned_data['macd'] = macd_indicator.macd()
cleaned_data['macd_signal'] = macd_indicator.macd_signal()

# Calculate rolling 20-day standard deviation of returns for volatility measurement
cleaned_data['roll_std_20'] = cleaned_data['Close'].pct_change().rolling(window=20).std()

# Drop any rows with NaN values created by indicator calculation
cleaned_data.dropna(inplace=True)

# Display the first few rows of the dataset with the new features
cleaned_data.head()


In [ ]:
# Calculate 10-day and 50-day Simple Moving Averages
cleaned_data['ma_10'] = cleaned_data['Close'].rolling(window=10).mean()
cleaned_data['ma_50'] = cleaned_data['Close'].rolling(window=50).mean()

# Calculate Relative Strength Index (RSI) with a 14-day window
delta = cleaned_data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.rolling(window=14, min_periods=1).mean()
avg_loss = loss.rolling(window=14, min_periods=1).mean()

rs = avg_gain / avg_loss
cleaned_data['rsi_14'] = 100 - (100 / (1 + rs))

# Calculate Bollinger Bands (20-day MA ± 2σ)
cleaned_data['ma_20'] = cleaned_data['Close'].rolling(window=20).mean()
cleaned_data['bb_upper'] = cleaned_data['ma_20'] + 2 * cleaned_data['Close'].rolling(window=20).std()
cleaned_data['bb_lower'] = cleaned_data['ma_20'] - 2 * cleaned_data['Close'].rolling(window=20).std()

# Calculate Moving Average Convergence Divergence (MACD)
ema_12 = cleaned_data['Close'].ewm(span=12, adjust=False).mean()
ema_26 = cleaned_data['Close'].ewm(span=26, adjust=False).mean()
cleaned_data['macd'] = ema_12 - ema_26
cleaned_data['macd_signal'] = cleaned_data['macd'].ewm(span=9, adjust=False).mean()

# Calculate rolling 20-day standard deviation of returns for volatility measurement
cleaned_data['roll_std_20'] = cleaned_data['Close'].pct_change().rolling(window=20).std()

# Drop any rows with NaN values created by indicator calculation
cleaned_data.dropna(inplace=True)

# Display the first few rows of the dataset with the new features
cleaned_data.head()


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import numpy as np

# Prepare features and target for Strategy A model
features = ['ma_10', 'ma_50', 'rsi_14', 'ma_20', 'bb_upper', 'bb_lower', 'macd', 'macd_signal', 'roll_std_20']
cleaned_data['return'] = cleaned_data['Close'].pct_change().shift(-1)  # Predict next-day return
cleaned_data.dropna(inplace=True)

X = cleaned_data[features]
y = cleaned_data['return']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Ridge Regression model
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

# Use the trained model to make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate model performance
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Display model performance metrics
mse, mae


In [ ]:
# Initialize variables for the trading simulation
initial_capital = 5000
capital = initial_capital
shares = 0
transaction_cost = 0.001  # 0.1% transaction cost
threshold_return = 0.001  # Threshold for trading signal (0.1%)

# Create a copy of the test set for the simulation
test_data = X_test.copy()
test_data['predicted_return'] = y_pred
test_data['close'] = cleaned_data.loc[test_data.index, 'Close']
test_data['equity'] = 0.0

# Simulate trading based on the predicted returns
for i in range(len(test_data)):
    # Get the current row of data
    current_row = test_data.iloc[i]
    predicted_return = current_row['predicted_return']
    current_price = current_row['close']
    
    # Trading decision based on predicted return
    if predicted_return > threshold_return:
        # Buy shares if not already holding any
        if shares == 0:
            shares_to_buy = int(capital / (current_price * (1 + transaction_cost)))
            if shares_to_buy > 0:
                capital -= shares_to_buy * current_price * (1 + transaction_cost)
                shares += shares_to_buy
    else:
        # Sell shares if holding any
        if shares > 0:
            capital += shares * current_price * (1 - transaction_cost)
            shares = 0

    # Update equity value
    test_data.iloc[i, test_data.columns.get_loc('equity')] = capital + (shares * current_price)

# Calculate final equity and performance metrics
final_equity = test_data['equity'].iloc[-1]
total_return = (final_equity - initial_capital) / initial_capital
annualized_return = ((1 + total_return) ** (1 / (len(test_data) / 252))) - 1  # Assuming 252 trading days in a year

# Display the results
final_equity, total_return, annualized_return


In [ ]:
# Initialize variables for Strategy B trading simulation
capital = initial_capital
shares = 0
test_data_b = cleaned_data.loc[X_test.index].copy()  # Use the same time period as Strategy A
test_data_b['position'] = 0
test_data_b['equity'] = 0.0

# Moving Average Crossover Strategy with Volatility Scaling
for i in range(len(test_data_b)):
    current_row = test_data_b.iloc[i]
    current_price = current_row['Close']
    st_ma = current_row['ma_10']
    lt_ma = current_row['ma_50']
    volatility = current_row['roll_std_20']
    
    # Volatility scaling factor (scale inversely to volatility)
    if volatility > 0:
        scale_factor = 0.001 / volatility
    else:
        scale_factor = 1.0
    
    # Cap scale factor to not exceed 1.0 (full allocation)
    scale_factor = min(scale_factor, 1.0)
    
    # Determine buy/sell signals
    if i > 0:
        prev_st_ma = test_data_b.iloc[i - 1]['ma_10']
        prev_lt_ma = test_data_b.iloc[i - 1]['ma_50']
        buy_signal = (prev_st_ma < prev_lt_ma) and (st_ma > lt_ma)
        sell_signal = (prev_st_ma > prev_lt_ma) and (st_ma < lt_ma)
    else:
        buy_signal = False
        sell_signal = False

    # Execute trading decisions
    if buy_signal:
        # Buy with volatility-scaled position size
        if shares == 0:
            invest_amount = capital * scale_factor
            shares_to_buy = int(invest_amount / (current_price * (1 + transaction_cost)))
            if shares_to_buy > 0:
                cost = shares_to_buy * current_price * (1 + transaction_cost)
                capital -= cost
                shares += shares_to_buy
                
    if sell_signal and shares > 0:
        # Sell all shares
        capital += shares * current_price * (1 - transaction_cost)
        shares = 0

    # Update equity value
    test_data_b.iloc[i, test_data_b.columns.get_loc('equity')] = capital + (shares * current_price)

# Calculate final equity and performance metrics for Strategy B
final_equity_b = test_data_b['equity'].iloc[-1]
total_return_b = (final_equity_b - initial_capital) / initial_capital
annualized_return_b = ((1 + total_return_b) ** (1 / (len(test_data_b) / 252))) - 1  # Assuming 252 trading days in a year

# Display the results
final_equity_b, total_return_b, annualized_return_b
